<a href="https://colab.research.google.com/github/Samin-Sadaf7/MasteringObjectDetection/blob/main/MateringObhectDetection_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bird Mulitple object Detection using FasterRCNN and Tensorflow Object Detection API

Part -2 : training

### Download model from tensorflow 2 model garden

In [2]:
import os
import re #regular expression
import tensorflow as tf
from google.colab import drive
#Mounting to Google Drive
drive.mount('/content/gdrive', force_remount=True)
#Set current directory to MyDrive , put the downloaded files there
%cd gdrive/MyDrive/
# Get current working dir
current_dir= os.getcwd()
print("Root dir:", current_dir)

Mounted at /content/gdrive
/content/gdrive/MyDrive
Root dir: /content/gdrive/MyDrive


In [3]:
#Define some constants
RANDOM_SEED = 99
BATCH_SIZE = 1
NUM_STEPS = 2100
NUM_EVAL_STEPS = 1000

In [4]:
# Check if the directory exists
if not os.path.isdir('faster_rcnn_resnet101_v1_640x640_coco17_tpu-8'):
    # Download the model
    !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
    # Extract the compressed file
    !tar -xf faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
    # Remove the compressed file
    !rm faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz

##Generate label map

In [12]:
labels = [
    "Indian Robinrotation",
    "Oriental Magpie Robinrotation",
    "Alexandrine Parakeetrotation",
    "Asian Koelrotation",
    "Indian Paradise-Flycatcherrotation",
    "Laughing Doverotation",
    "Indian Spot-billed Duckrotation",
    "Asian Wooly-necked storkrotation",
    "Bulbulrotation",
    "Barn owlrotation",
    "Woodpeakerrotation",
    "Yellow breasted greenfinchrotation",
    "Red Bearded Bea Eaterrotation",
    "Wrenrotation",
    "Black Kiterotation",
    "Brahminy Kiterotation"
]

with open('label_map.pbtxt', 'w') as f:
    for i, label in enumerate(labels, 1):
        name = label.lower().replace(' ', '_')
        f.write('item {\n')
        f.write('\tname: "{}"\n'.format(name))
        f.write('\tid: {}\n'.format(i))
        f.write('\tdisplay_name: "{}"\n'.format(label))
        f.write('}\n')

### Convert XML annotations to CSV file

In [14]:
!rm xml_to_csv.py
%pwd
#Download the latest version from github
!wget https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/xml_to_csv.py

rm: cannot remove 'xml_to_csv.py': No such file or directory
--2024-04-07 23:51:35--  https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/xml_to_csv.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1291 (1.3K) [text/plain]
Saving to: ‘xml_to_csv.py’

xml_to_csv.py       100%[===================>]   1.26K  --.-KB/s    in 0s      

2024-04-07 23:51:35 (64.2 MB/s) - ‘xml_to_csv.py’ saved [1291/1291]



In [19]:
#train labels csv
!python xml_to_csv.py --xml_folder $current_dir/birdsod/train --output_csv $current_dir/birdsod/train/train_labels.csv

In [21]:
#test labels csv
!python xml_to_csv.py --xml_folder $current_dir/birdsod/test --output_csv $current_dir/birdsod/test/test_labels.csv

##Convert csv to TFrecords

In [10]:
#Get the latest version , remove the older
!rm generate_tfrecord.py
#Download the latest version from github
!wget https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/generate_tfrecord.py

rm: cannot remove 'generate_tfrecord.py': No such file or directory
--2024-04-08 00:30:24--  https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/generate_tfrecord.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4117 (4.0K) [text/plain]
Saving to: ‘generate_tfrecord.py’

generate_tfrecord.p 100%[===================>]   4.02K  --.-KB/s    in 0.002s  

2024-04-08 00:30:25 (2.07 MB/s) - ‘generate_tfrecord.py’ saved [4117/4117]



In [13]:
!python generate_tfrecord.py \
  --csv_input=/content/gdrive/MyDrive/birdsod/train/train_labels.csv \
  --image_dir=/content/gdrive/MyDrive/birdsod/train\
  --output_path=/content/gdrive/MyDrive/birdsod/train/train.record

2024-04-08 00:49:27.924204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Successfully created the TFRecords: /content/gdrive/MyDrive/birdsod/train/train.record


In [14]:
!python generate_tfrecord.py \
  --csv_input=/content/gdrive/MyDrive/birdsod/test/test_labels.csv \
  --image_dir=/content/gdrive/MyDrive/birdsod/test \
  --output_path=/content/gdrive/MyDrive/birdsod/test/test.record

2024-04-08 00:51:09.531669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Successfully created the TFRecords: /content/gdrive/MyDrive/birdsod/test/test.record


In [17]:
current_dir

'/content/gdrive/MyDrive'

In [18]:
#train test record path
train_record_path = os.path.join(current_dir,'birdsod','train','train.record')
test_record_path = os.path.join(current_dir,'birdsod','test','test.record')
label_map_path = '/content/gdrive/MyDrive/models/research/label_map.pbtxt'

####Config files, checkpoints and hyperparameter

In [19]:
#Load the last checkpoint if it exists
fine_tune_checkpoint_fastercnn = 'faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0'
print('Checkpoint Dir for FasterRCNN:', fine_tune_checkpoint_fastercnn)

Checkpoint Dir for FasterRCNN: faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0


In [20]:
#Hyperparameters
if os.path.isfile('pipeline.config'):
  !rm 'pipeline.config'

#Download the pipeline
!wget https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/pipeline.config

#set config file variables
base_config_path_fasterrcnn= 'pipeline.config'

--2024-04-08 01:22:42--  https://raw.githubusercontent.com/ayoubbensakhria/ObjectDetectionFRCNN/master/faster_rcnn_resnet101/pipeline.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3834 (3.7K) [text/plain]
Saving to: ‘pipeline.config’

pipeline.config     100%[===================>]   3.74K  --.-KB/s    in 0.003s  

2024-04-08 01:22:42 (1.16 MB/s) - ‘pipeline.config’ saved [3834/3834]



###Hyperparameters


In [21]:
import re

def edit_config(model_name, base_config_path, fine_tune_checkpoint):
    with open(base_config_path) as f:
        config = f.read()
    with open('{model}_config.config'.format(model=model_name), 'w') as f:
        #set labelmap path
        config = re.sub('label_map_path: ".*?"',
                        'label_map_path: "{}"'.format(label_map_path), config)

        #set fine tune checkpoint path
        config = re.sub('fine_tune_checkpoint: ".*?"',
                    'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

        #set train tf-record file path
        config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
                     'input_path: "{}"'.format(train_record_path), config)

        #set test tf-record file path
        config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/test)(.*?")',
                     'input_path: "{}"'.format(test_record_path), config)

        #set number of classes
        config = re.sub('num_classes: [0-9]+',
                    'num_classes: {}'.format(4), config)

        #set batch size
        config = re.sub('batch_size: [0-9]+',
                    'batch_size: {}'.format(BATCH_SIZE), config)

        #set training steps
        config = re.sub('num_steps: [0-9]+',
                    'num_steps: {}'.format(NUM_STEPS), config)

        #set up finetune checkpoint type to detection
        config = re.sub('fine_tune_checkpoint_type: "classification"',
                'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
        f.write(config)


In [22]:
#Edit config of faster rcnn using the above function
edit_config('fasterrcnn', base_config_path_fasterrcnn, fine_tune_checkpoint_fastercnn)

#Clean up
!rm 'pipeline.config'

#print pipeline config
%cat 'fasterrcnn_config.config'

# Faster R-CNN with Resnet-101 (v1)
# Trained on COCO, initialized from Imagenet classification checkpoint

# This config is TPU compatible.

model {
  faster_rcnn {
    num_classes: 4
    image_resizer {
      fixed_shape_resizer {
        width: 640
        height: 640
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_los